# Agentic Trading System for ETFs and Major Asset Classes
Team:

Zien Zhu (zienzhu2)

Yongpeng Fu (yf10)

Bowen Wang (bwang10)

Fazeh Aghajani (faezeha2)

Yiming Tong (yimingt7)

Yijie Wang (yijiew6)




In [ ]:
!pip install yfinance ta
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from datetime import timedelta

## SPY ETF: Large-Cap Index

**Data Collection Agent**

In [ ]:
# Data Collection Agent
def get_data(ticker, start="2005-01-01", end="2025-01-01"):
    df = yf.download(ticker, start=start, end=end)

    # Ensure "Adj Close" exists before dropping
    if "Adj Close" in df.columns:
        df.drop(columns=["Adj Close"], inplace=True)

    df.dropna(inplace=True)  # Handle missing data
    return df

# Fetch data for SPY (S&P 500 ETF)
data = get_data("SPY")

# Clean up the DataFrame
data.columns = data.columns.droplevel(1)  # Drop the "SPY" level in the column names
data = data.reset_index()  # Reset the index to make "Date" a column

# Display the cleaned DataFrame
data.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2005-01-03,82.316681,83.315702,82.042975,83.178847,55748000
1,2005-01-04,81.310799,82.480885,81.043937,82.426143,69167600
2,2005-01-05,80.749710,81.598193,80.742866,81.249218,65667300
3,2005-01-06,81.160278,81.529780,80.920788,81.043955,47814700
4,2005-01-07,81.043968,81.584536,80.831843,81.406626,55847700


**Analysis Agent**

In [ ]:
# Analysis Agent
def compute_indicators(df):
    df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    df["SMA_200"] = ta.trend.sma_indicator(df["Close"], window=200)
    df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df["MACD"] = ta.trend.macd(df["Close"])
    df["MACD_Signal"] = ta.trend.macd_signal(df["Close"])
    return df

data = compute_indicators(data)
data.head()

Price,Date,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI,MACD,MACD_Signal
0,2005-01-03,82.316681,83.315702,82.042975,83.178847,55748000,NaN,NaN,NaN,NaN,NaN
1,2005-01-04,81.310799,82.480885,81.043937,82.426143,69167600,NaN,NaN,NaN,NaN,NaN
2,2005-01-05,80.749710,81.598193,80.742866,81.249218,65667300,NaN,NaN,NaN,NaN,NaN
3,2005-01-06,81.160278,81.529780,80.920788,81.043955,47814700,NaN,NaN,NaN,NaN,NaN
4,2005-01-07,81.043968,81.584536,80.831843,81.406626,55847700,NaN,NaN,NaN,NaN,NaN


**Strategy Development Agent**

In [ ]:
def generate_weekly_signals(df):
    # Ensure 'Date' is in datetime format and set as index
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)

    # Calculate SMA indicators
    df["SMA_50"] = df["Close"].rolling(window=50).mean()
    df["SMA_200"] = df["Close"].rolling(window=200).mean()

    # Generate initial signals based on SMA crossovers
    df["Signal"] = 0
    df.loc[df["SMA_50"] > df["SMA_200"], "Signal"] = 1  # Buy signal
    df.loc[df["SMA_50"] < df["SMA_200"], "Signal"] = -1  # Sell signal

    # No holding period restriction, directly apply the signal
    df["Filtered_Signal"] = df["Signal"]
    return df

# Example usage:
data = generate_weekly_signals(data)
data.head()


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI,MACD,MACD_Signal,Signal,Filtered_Signal
Date,,,,,,,,,,,,
2005-01-03,82.316681,83.315702,82.042975,83.178847,55748000,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-04,81.310799,82.480885,81.043937,82.426143,69167600,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-05,80.749710,81.598193,80.742866,81.249218,65667300,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-06,81.160278,81.529780,80.920788,81.043955,47814700,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-07,81.043968,81.584536,80.831843,81.406626,55847700,NaN,NaN,NaN,NaN,NaN,0,0


**Portfolio Management Agent**

In [ ]:
# Portfolio Management Agent
def calculate_position_size(account_balance, risk_per_trade, stop_loss_pct):
    return (account_balance * risk_per_trade) / stop_loss_pct

position_size = calculate_position_size(100000, 0.02, 0.05)
print("Position Size:", position_size)

Position Size: 40000.0


**Backtesting Module**

In [ ]:
def backtest(df, initial_balance=10000):
    # Calculate daily returns based on the signal (shifted by 1 to simulate next day's trade)
    df["Daily Return"] = df["Close"].pct_change() * df["Filtered_Signal"].shift(1)

    # Calculate portfolio value based on the daily returns
    df["Portfolio Value"] = initial_balance * (1 + df["Daily Return"]).cumprod()

    # Fill NaN values in the Portfolio Value column (if any)
    df["Portfolio Value"].fillna(initial_balance, inplace=True)

    return df

# Apply backtest to data
backtested_data = backtest(data)

# Show the result
backtested_data[["Close", "Filtered_Signal", "Daily Return", "Portfolio Value"]].head()



<ipython-input-378-d81f30344215>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Portfolio Value"].fillna(initial_balance, inplace=True)


Price,Close,Filtered_Signal,Daily Return,Portfolio Value
Date,,,,
2005-01-03,82.316681,0,NaN,10000.0
2005-01-04,81.310799,0,-0.0,10000.0
2005-01-05,80.749710,0,-0.0,10000.0
2005-01-06,81.160278,0,0.0,10000.0
2005-01-07,81.043968,0,-0.0,10000.0


**Performance Metrics**

In [ ]:
import numpy as np

def performance_metrics(df, risk_free_rate=0.0164):
    # Convert annual risk-free rate to daily risk-free rate
    daily_risk_free_rate = (1 + risk_free_rate) ** (1 / 252) - 1

    # Calculate daily returns based on 'Portfolio Value'
    df['Daily Return'] = df['Portfolio Value'].pct_change()

    # Excess returns (daily return - risk-free rate)
    df["Excess Return"] = df["Daily Return"] - daily_risk_free_rate

    # Sharpe Ratio: The ratio of excess return to the standard deviation of return, annualized.
    sharpe_ratio = df["Excess Return"].mean() / df["Daily Return"].std() * np.sqrt(252)

    # Maximum Drawdown: The maximum loss from a peak to a trough of the portfolio.
    df['Cumulative Max'] = df['Portfolio Value'].cummax()
    df['Drawdown'] = df['Portfolio Value'] / df['Cumulative Max'] - 1
    max_drawdown = df['Drawdown'].min()

    # Win Rate: The proportion of trades where the daily return is positive.
    win_rate = (df['Daily Return'] > 0).sum() / len(df['Daily Return'])

    # Annualized Return: The average daily return compounded over 252 trading days.
    total_return = df['Portfolio Value'].iloc[-1] / df['Portfolio Value'].iloc[0] - 1
    annualized_return = (1 + total_return) ** (252 / len(df)) - 1

    # Trade Count: The number of buy and sell signals generated (i.e., non-zero signals)
    trade_count = (df['Filtered_Signal'].diff().abs() != 0).sum()

    return sharpe_ratio, max_drawdown, win_rate, annualized_return, trade_count

# Assuming backtested_data contains the daily returns and portfolio values
sharpe, drawdown, winrate, annualized_return, trade_count = performance_metrics(backtested_data)

# Print the results
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Max Drawdown: {drawdown:.2%}")
print(f"Win Rate: {winrate:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Trade Count: {trade_count}")
print(f"Net Profit: {(annualized_return+1)**20 - 1:.2f}")


Sharpe Ratio: 0.32
Max Drawdown: -40.12%
Win Rate: 52.08%
Annualized Return: 6.13%
Trade Count: 20
Net Profit: 2.29


## Invesco QQQ Trust (QQQ):  large-cap growth stocks in Tech industry

**Data Collection Agent**

In [ ]:
# Data Collection Agent
def get_data(ticker, start="2005-01-01", end="2025-01-01"):
    df = yf.download(ticker, start=start, end=end)

    # Ensure "Adj Close" exists before dropping
    if "Adj Close" in df.columns:
        df.drop(columns=["Adj Close"], inplace=True)

    df.dropna(inplace=True)  # Handle missing data
    return df

# Fetch data for QQQ (Nasdaq-100 ETF)
data2 = get_data("QQQ", start="2005-01-01", end="2025-01-01")

# Clean up the DataFrame
data2.columns = data2.columns.droplevel(1)  # Drop the "QQQ" level in the column names
data2 = data2.reset_index()  # Reset the index to make "Date" a column

# Display the cleaned DataFrame
data2.head()


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2005-01-03,33.866261,34.543587,33.754801,34.372111,100970900
1,2005-01-04,33.248966,34.072048,33.051771,34.012029,136623200
2,2005-01-05,33.043194,33.403289,32.983178,33.163226,127925500
3,2005-01-06,32.880287,33.188942,32.871715,33.120354,102934600
4,2005-01-07,33.051773,33.326133,32.760266,33.060349,123104000


**Analysis Agent**

In [ ]:
# Analysis Agent
def compute_indicators(df):
    df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    df["SMA_200"] = ta.trend.sma_indicator(df["Close"], window=200)
    df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df["MACD"] = ta.trend.macd(df["Close"])
    df["MACD_Signal"] = ta.trend.macd_signal(df["Close"])
    return df

data2 = compute_indicators(data2)
data2.head()

Price,Date,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI,MACD,MACD_Signal
0,2005-01-03,33.866261,34.543587,33.754801,34.372111,100970900,NaN,NaN,NaN,NaN,NaN
1,2005-01-04,33.248966,34.072048,33.051771,34.012029,136623200,NaN,NaN,NaN,NaN,NaN
2,2005-01-05,33.043194,33.403289,32.983178,33.163226,127925500,NaN,NaN,NaN,NaN,NaN
3,2005-01-06,32.880287,33.188942,32.871715,33.120354,102934600,NaN,NaN,NaN,NaN,NaN
4,2005-01-07,33.051773,33.326133,32.760266,33.060349,123104000,NaN,NaN,NaN,NaN,NaN


**Strategy Development Agent**

In [ ]:
# Gaussian Channel + Bollinger Bands Strategy
def generate_signals_gaussian_bollinger(df, window=50, bollinger_factor=2, threshold=2):
    # Ensure 'Date' is in datetime format and set as index
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)

    # Calculate rolling mean and standard deviation for Gaussian Channel
    df["Rolling Mean"] = df["Close"].rolling(window=window).mean()
    df["Rolling Std"] = df["Close"].rolling(window=window).std()

    # Define Gaussian Channel (upper and lower bounds)
    df["Gaussian Upper"] = df["Rolling Mean"] + threshold * df["Rolling Std"]
    df["Gaussian Lower"] = df["Rolling Mean"] - threshold * df["Rolling Std"]

    # Calculate Bollinger Bands
    df["Bollinger Upper"] = df["Rolling Mean"] + bollinger_factor * df["Rolling Std"]
    df["Bollinger Lower"] = df["Rolling Mean"] - bollinger_factor * df["Rolling Std"]

    # Generate signals based on Bollinger Bands and Gaussian Channel
    df["Signal"] = 0
    # Buy when price is below both the Bollinger Lower Band and Gaussian Lower
    df.loc[(df["Close"] < df["Bollinger Lower"]) & (df["Close"] < df["Gaussian Lower"]), "Signal"] = 1
    # Sell when price is above both the Bollinger Upper Band and Gaussian Upper
    df.loc[(df["Close"] > df["Bollinger Upper"]) & (df["Close"] > df["Gaussian Upper"]), "Signal"] = -1

    # Trading restriction logic (only trade on Mondays)
    last_trade_day = None  # Track the last trade date
    min_holding_days = 5   # Minimum holding period of 5 days

    signals = []  # Store final trading signals
    for i in range(len(df)):
        current_date = df.index[i]

        if last_trade_day is None or (current_date - last_trade_day).days >= min_holding_days:
            # Execute trades only on Mondays (weekday == 0)
            if current_date.weekday() == 0:
                signals.append(df["Signal"].iloc[i])
                if df["Signal"].iloc[i] != 0:
                    last_trade_day = current_date  # Update trade date
            else:
                signals.append(0)  # No trades on other days
        else:
            signals.append(0)  # No trades within the holding period

    df["Filtered_Signal"] = signals
    return df

# Apply Gaussian Channel + Bollinger Bands strategy to QQQ data
data2 = generate_signals_gaussian_bollinger(data2)
data2.head()


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI,MACD,MACD_Signal,Rolling Mean,Rolling Std,Gaussian Upper,Gaussian Lower,Bollinger Upper,Bollinger Lower,Signal,Filtered_Signal
Date,,,,,,,,,,,,,,,,,,
2005-01-03,33.866261,34.543587,33.754801,34.372111,100970900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-04,33.248966,34.072048,33.051771,34.012029,136623200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-05,33.043194,33.403289,32.983178,33.163226,127925500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-06,32.880287,33.188942,32.871715,33.120354,102934600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2005-01-07,33.051773,33.326133,32.760266,33.060349,123104000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


**Portfolio Management Agent**

In [ ]:
# Portfolio Management Agent
def calculate_position_size(account_balance, risk_per_trade, stop_loss_pct):
    return (account_balance * risk_per_trade) / stop_loss_pct

# Assume account balance is 100,000, risk per trade is 2%, and stop loss percentage is 5% for QQQ
position_size_qqq = calculate_position_size(100000, 0.04, 0.05)  # Lower risk per trade and higher stop loss for QQQ
print("Position Size for QQQ:", position_size_qqq)

Position Size for QQQ: 80000.0


**Backtesting Module**

In [ ]:
# Backtesting Module for QQQ
def backtest(df, initial_balance=10000, position_size=None):
    # Calculate daily returns based on the signal (shifted by 1 to simulate next day's trade)
    df["Daily Return"] = df["Close"].pct_change() * df["Filtered_Signal"].shift(1)  # Use Filtered_Signal here for QQQ

    # If position_size is provided, adjust the daily return by the position size
    if position_size is not None:
        # Scale the daily return by the position size / account balance
        df["Daily Return"] *= position_size / initial_balance

    # Calculate portfolio value based on the daily returns
    df["Portfolio Value"] = initial_balance * (1 + df["Daily Return"]).cumprod()

    return df

# Apply backtest to QQQ data (assumed data2 is the cleaned QQQ data with the signal column)
backtested_data_qqq = backtest(data2, initial_balance=100000, position_size=position_size_qqq)  # Replace `data` with `data2` for QQQ
print("Backtest Completed for QQQ")
backtested_data_qqq.head()


Backtest Completed for QQQ


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI,MACD,MACD_Signal,...,Rolling Std,Gaussian Upper,Gaussian Lower,Bollinger Upper,Bollinger Lower,Signal,Filtered_Signal,Daily Return,Portfolio Value,Excess Return
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-03,33.866261,34.543587,33.754801,34.372111,100970900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
2005-01-04,33.248966,34.072048,33.051771,34.012029,136623200,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,-0.0,100000.0,-0.000065
2005-01-05,33.043194,33.403289,32.983178,33.163226,127925500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,-0.0,100000.0,-0.000065
2005-01-06,32.880287,33.188942,32.871715,33.120354,102934600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,-0.0,100000.0,-0.000065
2005-01-07,33.051773,33.326133,32.760266,33.060349,123104000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0.0,100000.0,-0.000065


**Performance Metrics**

In [ ]:
# Performance Metrics for QQQ
def performance_metrics(df, risk_free_rate=0.0164):
    # Convert annual risk-free rate to daily risk-free rate
    daily_risk_free_rate = (1 + risk_free_rate) ** (1 / 252) - 1

    # Calculate excess returns
    df["Excess Return"] = df["Daily Return"] - daily_risk_free_rate

    # Sharpe ratio: The ratio of excess return to standard deviation of return, annualized.
    sharpe_ratio = df["Excess Return"].mean() / df["Daily Return"].std() * np.sqrt(252)

    # Max drawdown: The maximum loss from a peak to a trough of the portfolio.
    max_drawdown = (df["Portfolio Value"].cummax() - df["Portfolio Value"]).max()

    # Win rate: The proportion of days where the daily return is positive.
    win_rate = (df["Daily Return"] > 0).sum() / len(df["Daily Return"])

    # Annualized return: The average daily return compounded over 252 trading days.
    total_return = df["Portfolio Value"].iloc[-1] / df["Portfolio Value"].iloc[1] - 1
    annualized_return = (1 + total_return) ** (252 / len(df)) - 1

    # Trade count: The number of buy and sell signals generated (using Filtered_Signal for QQQ)
    trade_count = (df["Filtered_Signal"] != 0).sum()

    return sharpe_ratio, max_drawdown, win_rate, annualized_return, trade_count

# Assuming backtested_data_qqq contains the daily returns and portfolio values for QQQ
sharpe_qqq, drawdown_qqq, winrate_qqq, annualized_return_qqq, trade_count_qqq = performance_metrics(backtested_data_qqq)

# Print the results for QQQ
print(f"Sharpe Ratio for QQQ: {sharpe_qqq:.2f}")
print(f"Max Drawdown for QQQ: {drawdown_qqq:.2f}")
print(f"Win Rate for QQQ: {winrate_qqq:.2%}")
print(f"Annualized Return for QQQ: {annualized_return_qqq:.2%}")
print(f"Trade Count for QQQ: {trade_count_qqq}")
print(f"Net Profit for QQQ: {(annualized_return_qqq+1)**20 - 1:.2f}")

Sharpe Ratio for QQQ: 0.31
Max Drawdown for QQQ: 5381.52
Win Rate for QQQ: 1.33%
Annualized Return for QQQ: 2.75%
Trade Count for QQQ: 121
Net Profit for QQQ: 0.72
